# save each cofactor as a separate file

In [1]:
from osgeo import gdal
import os
import re

# === Input: path to your multi-band raster ===
path = "../Data/UKCEH/OneDrive_2_26-07-2024/Appendix2_Spatial_data_layers/Data/"
tif = "Stack_ver5_010324.tif"
input_tif = os.path.join(path, tif)

# === Output directory for individual bands ===
folder = 'cofactors'
output_dir = os.path.join(path, folder)
os.makedirs(output_dir, exist_ok=True)

# === Open the multi-band raster ===
ds = gdal.Open(input_tif)
if ds is None:
    raise RuntimeError(f"Cannot open file: {input_tif}")

# === Loop over each band ===
for i in range(1, ds.RasterCount + 1):
    band = ds.GetRasterBand(i)

    # === Get the description (e.g., 'Bio12') ===
    desc = band.GetDescription()

    if not desc:
        desc = f"Band{i}"  # fallback if no description
    else:
        # Clean description for filename safety
        desc = re.sub(r"[^\w\-\.]", "_", desc.strip())

    # === Output file path ===
    out_path = os.path.join(output_dir, f"{desc}.tif")

    # === Extract and save using gdal.Translate ===
    gdal.Translate(
        out_path,
        ds,
        bandList=[i]
    )

    print(f"Saved band {i} as {desc}.tif")

print("✅ All bands extracted by description.")


Saved band 1 as Elevation_dm.tif
Saved band 2 as Slope_pc_x10.tif
Saved band 3 as TWI.tif
Saved band 4 as MRVBF.tif
Saved band 5 as Bio1.tif
Saved band 6 as Bio12.tif
Saved band 7 as NGD5.tif
Saved band 8 as Land_cover.tif
Saved band 9 as Landsat_blue_2000_Q1.tif
Saved band 10 as Landsat_blue_2000_Q2.tif
Saved band 11 as Landsat_blue_2000_Q3.tif
Saved band 12 as Landsat_blue_2000_Q4.tif
Saved band 13 as Landsat_green_2000_Q1.tif
Saved band 14 as Landsat_green_2000_Q2.tif
Saved band 15 as Landsat_green_2000_Q3.tif
Saved band 16 as Landsat_green_2000_Q4.tif
Saved band 17 as Landsat_red_2000_Q1.tif
Saved band 18 as Landsat_red_2000_Q2.tif
Saved band 19 as Landsat_red_2000_Q3.tif
Saved band 20 as Landsat_red_2000_Q4.tif
Saved band 21 as Landsat_NIR_2000_Q1.tif
Saved band 22 as Landsat_NIR_2000_Q2.tif
Saved band 23 as Landsat_NIR_2000_Q3.tif
Saved band 24 as Landsat_NIR_2000_Q4.tif
Saved band 25 as Landsat_SWIR1_2000_Q1.tif
Saved band 26 as Landsat_SWIR1_2000_Q2.tif
Saved band 27 as Landsat

# change Peat nan to 0 values

In [4]:
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling

def align_and_replace_nan(target_path, reference_path, output_path, replacement_value=0):
    """
    Replace NaN values in target raster with a specified value where reference raster has valid (non-NaN, non-nodata) values.
    
    Args:
        target_path (str): Path to target raster (e.g., Peat.tif).
        reference_path (str): Path to reference raster (e.g., Elevation_dm.tif).
        output_path (str): Path to save modified raster.
        replacement_value (float or int): Value to replace NaN (default: 0).
    """
    # Open reference raster
    with rasterio.open(reference_path) as ref_src:
        ref_data = ref_src.read(1)
        ref_profile = ref_src.profile
        ref_nodata = ref_src.nodatavals[0] if ref_src.nodatavals else None
        ref_transform = ref_src.transform
        ref_shape = ref_data.shape
    
    # Open target raster
    with rasterio.open(target_path) as tgt_src:
        tgt_data = tgt_src.read(1)
        tgt_nodata = tgt_src.nodatavals[0] if tgt_src.nodatavals else None
        tgt_transform = tgt_src.transform
    
    # Align target raster to reference raster's extent, resolution, and projection
    aligned_data = np.empty(ref_shape, dtype=tgt_data.dtype)
    reproject(
        source=tgt_data,
        destination=aligned_data,
        src_transform=tgt_transform,
        src_crs=tgt_src.crs,
        dst_transform=ref_transform,
        dst_crs=ref_src.crs,
        resampling=Resampling.nearest  # Use nearest for categorical data like Peat
    )
    
    # Create mask for valid values in reference raster (non-NaN and non-nodata)
    ref_valid_mask = np.isfinite(ref_data)
    if ref_nodata is not None:
        ref_valid_mask = ref_valid_mask & (ref_data != ref_nodata)
    
    # Replace NaN values in aligned target raster with replacement_value where reference is valid
    nan_mask = np.isnan(aligned_data)
    replace_mask = nan_mask & ref_valid_mask
    aligned_data[replace_mask] = replacement_value
    
    # Preserve original nodata regions in target raster
    if tgt_nodata is not None:
        if ref_nodata is not None:
            aligned_data[~ref_valid_mask] = tgt_nodata
        else:
            aligned_data[~ref_valid_mask] = tgt_nodata if np.isnan(aligned_data[~ref_valid_mask]).all() else aligned_data[~ref_valid_mask]
    
    # Update profile for output
    out_profile = ref_profile.copy()
    out_profile.update(
        dtype=rasterio.float32 if np.issubdtype(aligned_data.dtype, np.floating) else aligned_data.dtype,
        nodata=tgt_nodata if tgt_nodata is not None else -9999
    )
    
    # Write the modified raster
    with rasterio.open(output_path, 'w', **out_profile) as dst:
        dst.write(aligned_data, 1)
    
    print(f"Modified raster saved to {output_path}")

if __name__ == "__main__":
    path_data = '../Data/predictData/cofactors_all/Peat.tif'
    path_ref = '../Data/predictData/Elevation_dm.tif'
    output_path = '../Data/predictData/cofactors_used/Peat.tif'   
   
    
    align_and_replace_nan(path_data, path_ref, output_path, 0)

Modified raster saved to ../Data/predictData/cofactors_used/Peat.tif


# extract labelled data for model training using predictor values at the laballed soil samples

In [5]:
import glob
import rasterio
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from rasterio.sample import sample_gen

shp_path = '../Data/trainingData/trainshape/soil_class_12.shp'
path = '../Data/predictData/cofactors_used'
rasters = glob.glob(os.path.join(path, '*.tif'))

# Read shapefile
gdf = gpd.read_file(shp_path)
gdf = gdf.to_crs(epsg=27700)
    
# Store results
df = pd.DataFrame()
# Loop through rasters
for rfile in rasters:
    name = os.path.basename(rfile)[:-4]
    with rasterio.open(rfile) as src:
        # Get coordinates from shapefile
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

        # Extract raster values at those points
        values = list(src.sample(coords))
        values = np.array(values).flatten()
        df[name] = values
df['target'] = gdf['target']  
# Add x and y coordinates
df['EAST'] = gdf.geometry.x
df['NORTH'] = gdf.geometry.y

df.head()

,AAR,aspect,ASR,AT0,ATS,Bio1,Bio12,DROCK50,Elevation_dm,FCD,...,nmds_Profile_depth_x_50m,nmds_Profile_depth_y_50m,nmds_Texture_group_x_50m,nmds_Texture_group_y_50m,Slope_pc_x10,Texture_group,TWI,target,EAST,NORTH
0,738.374695,249.443954,365.979279,1423.118408,2359.743896,10.010938,801.503113,35.0,903.0,171.0,...,0.454204,0.092794,0.051178,0.166560,14.0,4.0,10.666275,10.0,405130.0,198110.0
1,741.771606,221.088425,367.579193,1420.104126,2356.623779,9.993813,801.634399,35.0,958.0,171.0,...,0.221442,-0.061536,0.181206,0.207153,43.0,4.0,9.268817,10.0,405260.0,198210.0
2,738.643921,260.676392,366.190826,1422.481934,2359.263916,10.014563,800.826904,35.0,904.0,171.0,...,0.467239,0.164364,0.023530,0.128950,14.0,4.0,9.726226,10.0,405170.0,198010.0
3,749.723267,229.763641,371.507294,1412.348633,2348.920166,9.982938,801.233154,35.0,998.0,173.0,...,0.179147,0.102563,0.325465,0.138121,27.0,4.0,8.854306,10.0,405400.0,198210.0
4,750.368103,231.203445,371.825226,1411.663574,2348.255127,9.979313,801.099365,35.0,1004.0,173.0,...,0.277259,0.078239,0.392877,0.143265,20.0,4.0,9.393635,5.0,405460.0,198210.0


In [6]:
# df['Peat'] = df['Peat'].fillna(0)
df.to_csv(os.path.join('../Data/trainingData/trainingSample.csv'), index=False)

# Additional training samples from Peat data

In [3]:
import glob
import rasterio
import pandas as pd
import numpy as np
import os
import geopandas as gpd

# Paths
path = '../Data/predictData/cofactors_used'
rasters = glob.glob(os.path.join(path, '*.tif'))

def extract_training_samples(shp_path, rasters):
    """
    Extract raster values at points from a shapefile.
    Returns a DataFrame with raster values + target + coordinates.
    """
    # Read shapefile
    gdf = gpd.read_file(shp_path)
    gdf = gdf.to_crs(epsg=27700)
    
    df = pd.DataFrame()
    
    # Loop through rasters
    for rfile in rasters:
        name = os.path.basename(rfile)[:-4]
        with rasterio.open(rfile) as src:
            coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
            values = list(src.sample(coords))
            values = np.array(values).flatten()
            df[name] = values
    
    # Add target and coordinates
    df["target"] = gdf["target"]
    df["EAST"] = gdf.geometry.x
    df["NORTH"] = gdf.geometry.y
    
    return df

# Example usage with two shapefiles
shp1 = '../Data/trainingData/trainshape/soil_class_12.shp'
shp2 = '../Data/trainingData/trainshape/Peat_Matthew.shp'

df1 = extract_training_samples(shp1, rasters)
df2 = extract_training_samples(shp2, rasters)

# Combine all training samples
df_all = pd.concat([df1, df2], ignore_index=True)

# print(df_all.head())
print(df_all.shape)
print(df_all.columns)


(82324, 29)
Index(['AAR', 'Aggregate_texture', 'ALC_old', 'Aquifers', 'aspect', 'ASR',
       'AT0', 'ATS', 'bedrock_raster_50m', 'Bio1', 'Bio12', 'CaCO3_rank',
       'DROCK50', 'Elevation_dm', 'FCD', 'Landcover_LE', 'Landsat_NIR_2000_Q1',
       'Landsat_NIR_2000_Q2', 'Landsat_NIR_2000_Q3', 'Landsat_NIR_2000_Q4',
       'MRVBF', 'NGD5', 'Profile_depth', 'Slope_pc_x10', 'Texture_group',
       'TWI', 'target', 'EAST', 'NORTH'],
      dtype='object')


In [4]:
# df_all['Peat'] = df_all['Peat'].fillna(0)
df_all.to_csv(os.path.join('../Data/trainingData/trainingSample.csv'), index=False)

In [56]:
# print(df.isna().sum())
columns = df.columns
category_columns = ['Aggregate_texture', 'Aquifers', 'CaCO3_rank', 'Land_cover', 'Peat', 'Profile_depth', 'Texture_group', 'target']

print(f'shape of the dataframe before dropping nadata: {df.shape}')
df = df.dropna()
print(f'shape of the dataframe after dropping nadata: {df.shape}')

for column in category_columns:
    print(f'{column}:{len(df[column].unique())}')
    # print(f'{column}:{df[column].value_counts()}')

shape of the dataframe before dropping nadata: (62298, 43)
shape of the dataframe after dropping nadata: (62298, 43)
Aggregate_texture:9
Aquifers:7
CaCO3_rank:7
Land_cover:29
Peat:4
Profile_depth:5
Texture_group:20
target:13


# BGS bedrock data

In [6]:
import os
import geopandas as gpd

path = '../Data/predictData/OtherSources/BGS/BGS_Geology_625k_bedrock_gpkg'
data = '625k_V5_Geology_UK_EPSG27700.gpkg'

gdf = gpd.read_file(os.path.join(path, data))

# Create a mapping of unique RCS_D values to unique integer IDs
unique_mapping = {value: idx + 1 for idx, value in enumerate(gdf['RCS_D'].dropna().unique())}

# Assign the new BEDROCK column based on the mapping
gdf['BEDROCK'] = gdf['RCS_D'].map(unique_mapping)

# Display the first few rows to verify
print(gdf['BEDROCK'].unique())

C:\ProgramData\anaconda3\envs\oggm_env\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in '625k_V5_Geology_UK_EPSG27700.gpkg': '625k_V5_BEDROCK_Geology' (default), '625k_V5_DYKES_Geology', '625k_V5_FAULTS', '625k_V5_SUPERFICIAL_Geology', 'layer_styles'. Specify layer parameter to avoid this warning.
  result = read_func(


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86]


In [10]:
import rasterio 
from rasterio.features import rasterize
import numpy as np

# Define raster properties
resolution = 50  # 50-meter resolution
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
transform = rasterio.transform.from_origin(bounds[0], bounds[3], resolution, resolution)
height = int((bounds[3] - bounds[1]) / resolution)
width = int((bounds[2] - bounds[0]) / resolution)

# Prepare the geometry and BEDROCK values for rasterization
shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf['BEDROCK']) if not np.isnan(value)]

# Create an empty array for the raster
raster = np.zeros((height, width), dtype=rasterio.uint16)

# Rasterize the geometries with BEDROCK values
raster = rasterize(shapes, out=raster, transform=transform, fill=0, dtype=rasterio.uint16)

# Define metadata for the GeoTIFF
meta = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': rasterio.uint16,
    'crs': gdf.crs,
    'transform': transform
}

# Save the raster to a GeoTIFF file
with rasterio.open(os.path.join(path, 'bedrock_raster_50m.tif'), 'w', **meta) as dst:
    dst.write(raster, 1)

# Display basic info to verify
print(f"Raster saved as 'bedrock_raster_50m.tif' with dimensions {width}x{height} at 50m resolution.")

Raster saved as 'bedrock_raster_50m.tif' with dimensions 13113x24302 at 50m resolution.


In [1]:
import os
import geopandas as gpd

path = '../Data/predictData/OtherSources/LE'
data = 'Living_England_Habitat_Map_Phase4_-8921961978092281537.gpkg'

gdf = gpd.read_file(os.path.join(path, data))


C:\ProgramData\anaconda3\envs\oggm_env\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: This version of GeoPackage user_version=0x000028A0 (10400, v1.4.0) on '../Data/predictData/OtherSources/LE\Living_England_Habitat_Map_Phase4_-8921961978092281537.gpkg' may only be partially supported
  return ogr_read(


In [2]:
print(gdf.columns)

Index(['ID', 'A_pred', 'A_prob', 'B_pred', 'B_prob', 'SrcCode', 'ImagesSpr',
       'ImagesAut', 'GlobalID', 'geometry'],
      dtype='object')


In [3]:
# Create a mapping of unique RCS_D values to unique integer IDs
unique_mapping = {value: idx + 1 for idx, value in enumerate(gdf['A_pred'].dropna().unique())}

# Assign the new BEDROCK column based on the mapping
gdf['Landcover'] = gdf['A_pred'].map(unique_mapping)

# Display the first few rows to verify
print(gdf['Landcover'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


In [4]:
import rasterio 
from rasterio.features import rasterize
import numpy as np

# Define raster properties
resolution = 50  # 50-meter resolution
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
transform = rasterio.transform.from_origin(bounds[0], bounds[3], resolution, resolution)
height = int((bounds[3] - bounds[1]) / resolution)
width = int((bounds[2] - bounds[0]) / resolution)

# Prepare the geometry and BEDROCK values for rasterization
shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf['Landcover']) if not np.isnan(value)]

# Create an empty array for the raster
raster = np.zeros((height, width), dtype=rasterio.uint16)

# Rasterize the geometries with BEDROCK values
raster = rasterize(shapes, out=raster, transform=transform, fill=0, dtype=rasterio.uint16)

# Define metadata for the GeoTIFF
meta = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': rasterio.uint16,
    'crs': gdf.crs,
    'transform': transform
}

# Save the raster to a GeoTIFF file
with rasterio.open(os.path.join(path, 'Landcover_LE_50m.tif'), 'w', **meta) as dst:
    dst.write(raster, 1)

# Display basic info to verify
print(f"Raster saved as 'bedrock_raster_50m.tif' with dimensions {width}x{height} at 50m resolution.")

MemoryError: 

# BGS 1:50000 bedrock data

In [15]:
import geopandas as gpd

gdb_path = '../Data/predictData/OtherSources/BGS/50000_Geology.gdb'

layer = 'gb_50k_bedrock'
gdf = gpd.read_file(gdb_path, layer=layer)
print(gdf.columns)

Index(['LEX_WEB', 'LEX', 'LEX_D', 'LEX_RCS', 'RCS', 'RCS_X', 'RCS_D',
       'RCS_ORIGIN', 'RANK', 'BED_EQ_D', 'MB_EQ_D', 'FM_EQ_D', 'SUBGP_EQ_D',
       'GP_EQ_D', 'SUPGP_EQ_D', 'MAX_TIME_Y', 'MIN_TIME_Y', 'MAX_AGE',
       'MAX_EPOCH', 'MAX_SUBPER', 'MAX_PERIOD', 'MAX_ERA', 'MAX_EON',
       'BGSTYPE', 'LEX_RCS_I', 'LEX_RCS_D', 'BGSREF', 'MAP_SRC', 'MAP_WEB',
       'VERSION', 'RELEASED', 'NOM_SCALE', 'NOM_BGS_YR', 'UUID',
       'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')


In [16]:
# compute raster bounds and transform
import rasterio
from rasterio.transform import from_origin
import numpy as np

resolution = 50  # meters
minx, miny, maxx, maxy = gdf.total_bounds

width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

transform = from_origin(minx, maxy, resolution, resolution)


In [23]:
# Create integer codes for each RCS class
import pandas as pd

# Make a unique code for each category
unique_vals = gdf["RCS_ORIGIN"].unique()
codes = {val: i for i, val in enumerate(unique_vals, start=1)}

print(codes)   # this is your lookup table

# Add numeric code column to the GeoDataFrame
gdf["RCS_code"] = gdf["RCS_ORIGIN"].map(codes)


{'IGNEOUS': 1, 'IGNEOUS AND METAMORPHIC': 2, 'METAMORPHIC': 3, 'SEDIMENTARY': 4, 'SEDIMENTARY AND IGNEOUS': 5, 'MINERAL VEIN': 6, 'UNKNOWN': 7, 'METAMORPHIC AND SEDIMENTARY': 8}


In [24]:
# Prepare (geometry, value) pairs for rasterization

shapes = [
    (geom, value)
    for geom, value in zip(gdf.geometry, gdf["RCS_code"])
]


In [25]:
# rasterise
from rasterio.features import rasterize

raster = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,              # background value
    dtype='float32'      # or int32 depending on RCS
)


In [26]:
# save to geotif
out_path = "../Data/predictData/OtherSources/BGS/bedrock_RCS_ORIGIN_50m.tif"

with rasterio.open(
    out_path,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=raster.dtype,
    crs='EPSG:27700',
    transform=transform
) as dst:
    dst.write(raster, 1)

